In [4]:
from functools import partial
import os

!{sys.executable} -m pip install opencv-python
import cv2
from numpy import argmax, linalg, vstack
from sklearn.random_projection import GaussianRandomProjection


def load_images(folder):
    histograms = []
    for filename in os.listdir(folder)[:2]:
        abs_path = os.path.join(folder, filename)
        image = cv2.imread(abs_path, 0)
        histograms.append(
            #cv2.calcHist([image], [0], None, [256], [0, 256]).reshape((256,))
            cv2.calcHist([image], [0], None, [256], [0, 256]).T[0]
        )
    #print(histograms[0])
    return vstack(histograms)


def load_labels(filename):
    with open(filename) as f:
        return list(
            filter(
                None,
                f.read().split('\n')
            )
        )


def cos_sim(v1, v2):
    v1_dot = v1 * v1.T
    v2_dot = v2 * v2.T
    dot_product = v1 * v2.T
    return dot_product / (v1_dot * v2_dot)


def random_projection(histograms):
    transformer = GaussianRandomProjection(n_components=64)
    return transformer.fit_transform(histograms)


def classify(train_data, test_data, labels, reduce_func=random_projection):
    train_data = reduce_func(train_data)
    print(train_data)
    train_data_norms = linalg.norm(train_data, axis=1)
    print(train_data_norms)
    print(train_data_norms.shape)
    train_data /= train_data_norms
    test_data = reduce_func(test_data)
    test_data_norms = linalg.norm(test_data, axis=1)
    test_data /= test_data_norms
    for obj in test_data:
        knn(train_data, obj, k=3)


def knn(train_data, labels, obj):
    sims = train_data * obj.T
    print(sims)


train_labels = load_labels('./train.labels')
test_labels = load_labels('./test.labels')
histograms = load_images('/Users/dtang/data/pr2/train')

/bin/sh: {sys.executable}: command not found


ModuleNotFoundError: No module named 'cv2'